In [9]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [7]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=GOOGLE_API_KEY)

url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
loader = WebBaseLoader(url)
documents = loader.load()

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [5]:
template = """
    You are a knowledgeable AI assistant. Based on the following context, answer the user's question.

    Context:
    {context}

    Question: {question}

    Answer:"""
custom_prompt = PromptTemplate(
    input_variables=['context','question'],
    template=template
)

In [10]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7656\598012801.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0518 23:35:56.501000 7656 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [11]:
vectorstore = FAISS.from_documents(docs, embeddings)
retriver = vectorstore.as_retriever(search_kwargs={"k":3})

In [12]:
qa_chain = LLMChain(llm=llm,prompt=custom_prompt)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_7656\3217261023.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm,prompt=custom_prompt)


In [15]:
def run_custom_retrieval_qa(query):
    # 1. Retrieve docs
    docs = retriver.get_relevant_documents(query)
    # 2. Join context
    context = "\n\n".join([doc.page_content for doc in docs])
    # 3. Run chain
    answer = qa_chain.run({'context':context,'question':query})
    
    return answer,docs

In [16]:
query = "What is the Turing Test?"
answer, used_docs = run_custom_retrieval_qa(query)

print("Answer:")
print(answer)

print("\n---\nUsed Documents:")
for doc in used_docs:
    print(doc.page_content[:200])  # Print a preview


C:\Users\ASUS\AppData\Local\Temp\ipykernel_7656\3818667795.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run({'context':context,'question':query})


Answer:
The Turing test measures the ability of a machine to simulate human conversation.

---
Used Documents:
^ a b Turing's original publication of the Turing test in "Computing machinery and intelligence": Turing (1950)
Historical influence and philosophical implications: Haugeland (1985, pp. 6–9), Crevier 
Alan Turing wrote in 1950 "I propose to consider the question 'can machines think'?"[381] He advised changing the question from whether a machine "thinks", to "whether or not it is possible for machin
The Turing test can provide some evidence of intelligence, but it penalizes non-human intelligent behavior.[383]
